<a href="https://colab.research.google.com/github/t6lase01/LAB42/blob/master/CelebA_ProGAN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from IPython.terminal.embed import embed
from tensorflow.python.module.module import Module
from absl import logging

import imageio
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
tf.random.set_seed(0)

import tensorflow_hub as hub
import time


try: 
  from google.colab import files
except ImportError:
  print("Import error occured")
  pass

from IPython import display
from skimage import transform

latent_dim = 512

"""
Interpolate between two non-zero vectors that also do not both line through 
origin. First vector2 is normalised to same norm as vector1. And then iterpolate
between the two vectors in a hypersphere.
"""

def interpolate_hypersphere(vector1, vector2, num_steps):
  vector1_norm = tf.norm(vector1)
  vector2_norm = tf.norm(vector2)
  vector2_normalised = vector2 * (vector1_norm / vector2_norm)

  vectors = []
  for step in range(num_steps):
    interpolated = vector1 + (vector2_normalised) * step / (num_steps - 1)
    interpolated_norm = tf.norm(interpolated)
    interpolated_normalised = interpolated * (vector1_norm / interpolated_norm)
    vectors.append(interpolated_normalised)
  return tf.stack(vectors)

# Display an image
def show_image(image):
  image = tf.constant(image)
  image = tf.image.convert_image_dtype(image, tf.uint8)
  return PIL.Image.fromarray(image.numpy())

# From the set of images, create animation
def animate(images):
  images = np.array(images)
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images)
  return embed.embed_file('./animation.gif')

logging.set_verbosity(logging.ERROR)

